<a href="https://colab.research.google.com/github/DanielaManate/SentimentAnalysis_MADC_UBB/blob/master/SA3_2_Text_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# for tokenizing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from gensim.models.fasttext import FastText

from sklearn.decomposition import PCA

import pickle

import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
path = 'gdrive/My Drive/Colab Notebooks/Data/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# %env PYTHONHASHSEED=0

# Reading Dataset

In [4]:
# Reading the dataset
reviews = pd.read_csv(path + 'Input/3.input_data_prepped_bow.csv')
reviews.head()

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9
2,yGMCl0vYigshkXiZFIDTNw,Breakfast was delicious and waitress was aweso...,5,146,1,breakfast delicious waitress awesome steak egg...,"['breakfast', 'delicious', 'waitress', 'awesom...",15,breakfast delicious waitress awesome steak egg...,"['breakfast', 'delicious', 'waitress', 'awesom...",15
3,yGMCl0vYigshkXiZFIDTNw,"We have eaten here twice, for lunch. The first...",4,710,1,eaten twice lunch first time french dip mom fi...,"['eaten', 'twice', 'lunch', 'first', 'time', '...",75,eaten twice lunch first time french dip mom fi...,"['eaten', 'twice', 'lunch', 'first', 'time', '...",74
4,yGMCl0vYigshkXiZFIDTNw,"awesome service, great food. Buffy was an ama...",5,59,1,awesome service great food buffy amazing server,"['awesome', 'service', 'great', 'food', 'buffy...",7,awesome service great food amazing server,"['awesome', 'service', 'great', 'food', 'amazi...",6


In [0]:
# Convert text_prep to string to counteract issues later
reviews['text_prep'] = reviews['text_prep'].astype(str)
reviews['text_prep_lim'] = reviews['text_prep_lim'].astype(str)

# DTM BOW

In [0]:
# dtm_bow = pd.read_csv(path + 'Output/1.dtm_1_bow.csv')
# dtm_bow.shape
# dtm_bow.head()

# DTM TF-IDF
- TF = # of times word appears in doc / total # of words
- IDF = log (total # of docs / # of docs containing the word)
TF-IDF = TF * IDF

In [7]:
# max_features = If not None, build a vocabulary that only consider the top 
# max_features ordered by term frequency across the corpus.
vec = TfidfVectorizer(max_features = 6000)

tf_idf = vec.fit_transform(reviews.text_prep_lim)
review_words = vec.get_feature_names()

dtm_tf_idf = pd.DataFrame(tf_idf.toarray(), 
                          columns=review_words)

dtm_tf_idf.head()

,aaabar,aaron,abacus,ability,able,absolute,absolutely,absurd,abundance,abundant,ac,acai,accent,accept,acceptable,acceptance,access,accessible,accident,accidentally,accommodate,accommodating,accommodation,accomodate,accompaniment,accompany,accord,accordingly,accordion,account,accuracy,accurate,accuse,ace,ache,achieve,acidic,acknowledge,acknowledgement,acoustic,...,yen,yep,yes,yesterday,yet,yike,yo,yogurt,yoke,yolk,york,yorker,young,younger,youngster,youth,yr,yrs,yuck,yum,yumm,yumminess,yummm,yummmm,yummo,yummy,yup,zach,zack,zero,zesty,zillion,zipps,ziti,zoe,zone,zucchini,zuchinni,zupa,zupas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
dtm_tf_idf['food']

0       0.064668
1       0.168440
2       0.124776
3       0.000000
4       0.250735
          ...   
9360    0.044254
9361    0.000000
9362    0.188899
9363    0.045726
9364    0.133919
Name: food, Length: 9365, dtype: float64

In [9]:
reviews.text_prep_lim.iloc[250]

'third time drop star guess new spot care service wait food joke doubt come back third time charm'

In [10]:
dtm_tf_idf['food'].iloc[250]

0.08007433498536726

In [0]:
dtm_tf_idf.to_csv(path+ 'Output/2.dtm_2_tfidf.csv', index = False)

In [12]:
dtm_tf_idf.shape

(9365, 5999)

# Word Embeddings: FastText

In [0]:
word_tokenized_corpus = reviews.text_prep_lim.apply(word_tokenize).to_list()

In [0]:
# final_size = 60 
# number of dimensions => every word will be represented as a 60-dimensional vector

# window_size = 10
# number of words before and after the word

# min_word = 3
# if freq < min_word, the word will not be represented

In [15]:
%%time

model = FastText(size=60,
            window=10,
            min_count=3)

model.build_vocab(sentences=word_tokenized_corpus)

model.train(sentences=word_tokenized_corpus,
            total_examples = len(word_tokenized_corpus),
            epochs=50)

CPU times: user 9min 18s, sys: 619 ms, total: 9min 19s
Wall time: 4min 46s


In [16]:
# In the output above, you can see a 60-dimensional vector for the word "food"
print(model.wv['food'])

[-0.39533824  0.89962405 -0.23431101  2.183999   -1.0123696   0.7251429
 -0.7633083  -0.99415225 -0.4091712   1.3306352   2.3160312   0.62584573
  1.3067356   0.01034489 -1.3670917  -1.712783    0.13874884 -2.711136
  2.6284256   0.18479992 -3.3829958   1.2906892   0.96651787 -2.9190528
  3.395584    0.6374713  -0.7961603  -3.0411308  -1.4473084  -3.4819381
  0.16783382  0.9434194   0.6640484   3.3993654   2.0446858  -3.3351042
 -2.5293198  -0.37849608  1.1498294   0.89274573  3.2484522  -0.7918358
 -2.270894   -2.7534523   0.76220566 -1.1957324  -2.7524428   0.20968905
  0.7145438  -2.530197   -3.5665154   3.0314095  -1.1358212   0.41627657
  1.2206402  -0.0063074   3.2000809  -4.313783    1.7165395  -2.3097053 ]


In [17]:
model.wv.most_similar('order', topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('border', 0.8309881687164307),
 ('ordered', 0.7924515008926392),
 ('orderly', 0.7268147468566895),
 ('ordering', 0.6709415912628174),
 ('wrong', 0.65912926197052)]

In [18]:
model.wv.most_similar('great', topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('excellent', 0.8277909755706787),
 ('awesome', 0.8032246828079224),
 ('excellently', 0.7761113047599792),
 ('greatly', 0.7636477947235107),
 ('fantastic', 0.7589607238769531)]

In [19]:
# We can also find the cosine similarity between the vectors for any two words, 
# as shown below:

print(model.wv.similarity(w1='order', w2='wait'))
print(model.wv.similarity(w1='slow', w2='slowly'))
print(model.wv.similarity(w1='slow', w2='fast'))

0.3719096
0.7896392
0.3093512


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



## Save model

In [0]:
pickle.dump(model, open(path + 'Output/3.fast_text.pkl', 'wb' ))

# # Load pickle
# model = pickle.load(open(path + 'Output/3.fast_text.pkl', 'rb' ))

## Visualizing Word Similarities

In [0]:
final_list = ['food', 'place', 'good', 'go', 'order', 'get', 'great', 'time', 
              'come', 'service', 'back', 'like', 'would', 'try', 'one', 'make', 
              'love', 'take', 'say', 'chicken', 'wait', 'restaurant', 'eat', 'really', 
              'pizza', 'well', 'delicious', 'always', 'also', 
              'ask']

In [22]:
X = model.wv[final_list]
X

array([[-0.39533824,  0.89962405, -0.23431101, ..., -4.313783  ,
         1.7165395 , -2.3097053 ],
       [-1.4827476 , -0.9082429 , -0.7017739 , ..., -2.048017  ,
         1.971302  , -1.4086272 ],
       [ 1.6518326 , -3.6967683 ,  0.6287922 , ..., -0.85787123,
        -0.47097751,  1.0195314 ],
       ...,
       [ 1.4336418 ,  0.15497792, -0.3175075 , ..., -3.4110332 ,
        -2.2366493 ,  1.2977325 ],
       [-1.1929742 , -1.1470975 ,  3.547451  , ..., -0.7787845 ,
        -2.8248167 ,  0.6777681 ],
       [-0.2187512 , -1.704722  ,  7.694731  , ...,  1.6152898 ,
        -6.1599393 , -4.5764804 ]], dtype=float32)

In [0]:
pca = PCA(n_components=2).fit_transform(X)

In [24]:
fig = px.scatter(x=pca[:, 0], y=pca[:, 1],
                 text = final_list)
fig.update_traces(textposition='top center', 
                  textfont_size=14)
fig.show()